In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import statannot
from numpy import log10
import bioinfokit.analys
import statannot
import sklearn
from sklearn.metrics import r2_score as score_R2

In [ ]:
file = './source_data/annotation_files/cancer_gene_census.csv'
cancer_gene = pd.read_csv(file,encoding='latin-1')

In [ ]:
def hubgenes(Group,column,csv):
#    print(Group)
    path_dir = './source_data/cytoscape/'
    file_list = os.listdir(path_dir)
    
    hub = pd.DataFrame(columns=['Name'])

    n = 0
    for i in file_list:
        if i.startswith(Group) and i.endswith(csv):
            file = './source_data/cytoscape/%s' % i
            x = pd.read_csv(file,sep = ',',header=1)
 
            algo = i.split('_')[column]
            MAX = x['Score'].max()

            #Fractional percentile score
            x[algo] = (x['Score']/MAX)
            x = x[['Name',algo]]
            
            hub = pd.merge(hub,x,how='outer',on='Name')
                   
            n+=1

            
    return hub

In [ ]:
TC_DICT = {}
TC_TOTAL = pd.DataFrame()
TC_20 = pd.DataFrame()
for i in ['human_WGCNA_group2','human_WGCNA_group3','human_WGCNA_group6']:

    Hgenes = hubgenes(i,4,'top_node.csv')
    Hgenes = Hgenes.replace(np.nan, 0)
    Hgenes['total']= Hgenes.iloc[:,1:13].sum(axis=1)
    Hgenes = Hgenes[['Name','MCC']]

    new_name = 'pattern1' if i == 'WGCNA_group2' else 'pattern2' if i == 'WGCNA_group3' else 'pattern3'
    
    hgenes = Hgenes
    hgenes = hgenes.sort_values(by=['MCC'], ascending=[False])
    hgenes = hgenes.reset_index(drop=True)
    hgenes['Group'] = i
    
    hgenes = hgenes.reset_index(drop=False)
    hgenes['Rank'] = hgenes['index']+1
    hgenes = hgenes.rename(columns={'Name':'gene','MCC':'Score'})[['gene','Score','Group','Rank']]
    
    if i == 'human_WGCNA_group6':
        TC20 = hgenes[hgenes['Rank'].isin(range(1,20))]
    else:
        TC20 = hgenes[hgenes['Rank'].isin(range(1,21))]
        
    TC_TOTAL = pd.concat([TC_TOTAL,hgenes],axis=0)
    TC_20 = pd.concat([TC_20,TC20],axis=0)

hubgene_605 = TC_TOTAL
#hubgene_605.to_csv('./source_data/hubgene/hubgene_605.txt',sep='\t',index=False,header=True)

hubgene_20 = TC_20
#hubgene_20.to_csv('./source_data/hubgene/hubgene_20.txt',sep='\t',index=False,header=True)

hubgene_cancer = TC_20[TC_20['gene'].isin(cancer_gene['Gene Symbol'])]
#hubgene_cancer.to_csv('./source_data/hubgene/hubgene_cancer.txt',sep='\t',index=False,header=True)

# GTEx

In [ ]:
###
#os.system('wget https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt -P ./source_data/GTEX')
#os.system('wget https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt -P ./source_data/GTEX')
#os.system('wget https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz -P ./source_data/GTEX')
#os.system('gunzip ./source_data/GTEX/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz')

GTEX_sample = pd.read_csv('./source_data/GTEX/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt', sep = '\t', header = 0)
GTEX_subject = pd.read_csv('./source_data/GTEX/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt', sep = '\t', header = 0)
GTEX_subject = GTEX_subject.dropna(axis=0)

GTEX_gene_read = pd.read_csv('./source_data/GTEX/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct', sep = '\t', header = 2)
GTEX_gene_read = GTEX_gene_read.rename(columns={'Name':'Ensembl_ID','Description':'gene'})

###
A = GTEX_gene_read.iloc[:,0:2]
B = GTEX_gene_read.iloc[:,2:]

nm = bioinfokit.analys.norm()
nm.cpm(df=B)
cpm_df = nm.cpm_norm
cpm_df = np.log2(cpm_df+1)

GTEX_CPM_hsa = pd.concat([A,cpm_df],axis=1)

In [ ]:
#GTEX_sample_A
GTEX_sample_A = GTEX_sample[['SAMPID','SMTS','SMTSD','SMAFRZE']]
GTEX_sample_A.to_csv('./source_data/GTEX/SampleAttributesDS.txt',sep='\t',index=False,header=True)
GTEX_sample_A = GTEX_sample_A[(GTEX_sample_A['SMAFRZE'] == 'RNASEQ')]

#GTEX_sample_BL & TT
GTEX_sample_BL = GTEX_sample_A[(GTEX_sample_A['SMTS'] == 'Blood') & (GTEX_sample_A['SMAFRZE'] == 'RNASEQ')]
GTEX_sample_TT = GTEX_sample_A[(GTEX_sample_A['SMAFRZE'] == 'RNASEQ')]

#GTEX_sample_B
GTEX_sample_B = GTEX_sample_A[['SAMPID','SMTS','SMTSD']]

In [ ]:
GTEX_CPM_hsa = GTEX_CPM_hsa[~(GTEX_CPM_hsa['gene'].str.contains('^MIR'))]
GTEX_CPM_hsa['gene'] = GTEX_CPM_hsa['gene'].str.replace('MT-','',case=False)

In [ ]:
def GTEX(dt,tissue_name):
    GTEX = dt.sort_values(by=['AGE'], ascending=[True])
    GTEX = GTEX.rename(columns={'value':'CPM'})
    GTEX = GTEX.dropna(axis=0)
    G = GTEX[GTEX['SMTS']==tissue_name]
    G = G.reset_index(drop=True)
    return G


def data_merging(hub_TC,A,B,C,Con):

    if hub_TC.empty:
        GTEX_annot = C
        GTEX_sel = GTEX_annot[['gene']+A['SAMPID'].unique().tolist()]
        GTEX_sel = GTEX_sel.set_index(['gene'])
    else:
        GTEX_annot = pd.merge(hub_TC,C,how='inner',on='gene')
        GTEX_sel = GTEX_annot[['gene','Group','Rank']+A['SAMPID'].unique().tolist()]
        GTEX_sel = GTEX_sel.set_index(['gene','Group','Rank'])        
     
    GTEX_sel = GTEX_sel.T

    SPL = GTEX_sel.index.str.split('-')
    GTEX_sel['SUBJID'] = SPL.str[0]+'-'+SPL.str[1]
    GTEX_sel = GTEX_sel.reset_index(drop=False)

    GTEX_sel = GTEX_sel.set_index(['SUBJID','index'])
    GTEX_sel = GTEX_sel.T
    GTEX_sel = GTEX_sel.reset_index(drop=False)
    
    if hub_TC.empty:
        GTEX_sel_m = pd.melt(GTEX_sel, id_vars=['gene'])
    else:
        GTEX_sel_m = pd.melt(GTEX_sel, id_vars=['gene','Group','Rank'])

    GTEX = pd.merge(GTEX_subject,GTEX_sel_m,how='inner',on='SUBJID')
    GTEX = GTEX.rename(columns={'index':'SAMPID'})
    GTEX = pd.merge(GTEX,B,how='inner',on='SAMPID')

    GTEX = GTEX[GTEX['SEX']==1]

    if Con == 'Normal':
        GTEX = GTEX[(GTEX['DTHHRDY']==1) | (GTEX['DTHHRDY']==2)]
    if Con == 'Abnormal':
        GTEX = GTEX[(GTEX['DTHHRDY']==3) | (GTEX['DTHHRDY']==4)]
    if Con == 'N1':
        GTEX = GTEX[(GTEX['DTHHRDY']==1) | (GTEX['DTHHRDY']==2)]
    if Con == 'N2':
        GTEX = GTEX[(GTEX['DTHHRDY']==3)]
    if Con == 'N3':
        GTEX = GTEX[(GTEX['DTHHRDY']==4)]

    return GTEX



def GTEX_GRAPH(HUB_TC,tissue):

    for N in [1,2,3]:
        GTEX_TT_N = data_merging(HUB_TC,GTEX_sample_TT,GTEX_sample_B,GTEX_CPM_hsa,"N{}".format(N))
        globals()["N_{}".format(N)] = GTEX(GTEX_TT_N,tissue)
        globals()["N_{}".format(N)]['NA'] = "N{}".format(N)

        
    fig = plt.figure(figsize=(15,15))

    gs = gridspec.GridSpec(nrows=4, ncols=4, height_ratios=[0.2,1,1,1],width_ratios=[1,6,6,6],hspace=0.8)

    N  = ['20-29','30-39','40-49','50-59','60-69','70-79']
    A1 = ['20-29','40-49','50-59','60-69','70-79']
    A2 = ['30-39','40-49','50-59','60-69','70-79']
    
    N_b  = [('20-29','70-79')]
    A1_b = [('20-29','70-79')]
    A2_b = [('30-39','70-79')]
    
    for Num,od,bp in zip([1,2,3],[N,A1,A2],[N_b,A1_b,A2_b]):

        NN_2 =  globals()["N_{}".format(Num)][globals()["N_{}".format(Num)]['Group'] == 'human_WGCNA_group2']
        NN_3 =  globals()["N_{}".format(Num)][globals()["N_{}".format(Num)]['Group'] == 'human_WGCNA_group3']
        NN_4 =  globals()["N_{}".format(Num)][globals()["N_{}".format(Num)]['Group'] == 'human_WGCNA_group6']    
     
        ax1 = fig.add_subplot(gs[Num,1])
        sns.boxplot(data = NN_2, x= 'AGE', y= 'CPM',color='#66c2a5',ax=ax1,showfliers=False)
        plt.ylabel('')
        plt.yticks(fontsize=12)
        plt.xticks(fontsize=12)
        sns.despine()
        statannot.add_stat_annotation(ax1,data = NN_2, x= 'AGE', y= 'CPM', order = od,
                                      box_pairs = bp,
                                      test='Mann-Whitney',text_format='star',loc='outside',verbose=2)

        ax2 = fig.add_subplot(gs[Num,2])
        sns.boxplot(data = NN_3, x= 'AGE', y= 'CPM',color='#8da0cb',ax=ax2,showfliers=False)
        plt.ylabel('')
        plt.yticks(fontsize=12)
        plt.xticks(fontsize=12)
        sns.despine()
        statannot.add_stat_annotation(ax2,data = NN_3, x= 'AGE', y= 'CPM', order = od,
                                      box_pairs = bp,
                                      test='Mann-Whitney',text_format='star',loc='outside',verbose=2)

        ax3 = fig.add_subplot(gs[Num,3])
        sns.boxplot(data = NN_4, x= 'AGE', y= 'CPM',color='#deb887',ax=ax3,showfliers=False)
        plt.ylabel('')
        plt.yticks(fontsize=12)
        plt.xticks(fontsize=12)
        sns.despine()
        statannot.add_stat_annotation(ax3,data = NN_4, x= 'AGE', y= 'CPM', order = od,
                                      box_pairs = bp,
                                      test='Mann-Whitney',text_format='star',loc='outside',verbose=2)
        
        
        
    fig.add_subplot(gs[0,1])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, -1, 'PT_1',ha='center', size=20)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    
    fig.add_subplot(gs[0,2])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, -1, 'PT_2',ha='center', size=20)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    
    fig.add_subplot(gs[0,3])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, -1, 'PT_4',ha='center', size=20)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    
    fig.add_subplot(gs[1,0])
    plt.xticks([])
    plt.yticks([])
    plt.text(1,0.2,'Normal_logCPM',ha='right', size=15,rotation=90)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    
    fig.add_subplot(gs[2,0])
    plt.xticks([])
    plt.yticks([])
    plt.text(1,0.1,'illness_1_logCPM',ha='right', size=15,rotation=90)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    
    fig.add_subplot(gs[3,0])
    plt.xticks([])
    plt.yticks([])
    plt.text(1,0.1,'illness_2_logCPM',ha='right', size=15,rotation=90)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)

    plt.show()

GTEX_GRAPH(hubgene_20,'Blood')     #hub20
GTEX_GRAPH(hubgene_cancer,'Blood')     #hub_cancer

# MFA-MML

In [ ]:
GTEX_BL = data_merging(hubgene_20,GTEX_sample_BL,GTEX_sample_B,GTEX_CPM_hsa,'Normal')
GTEX_BL_Ab = data_merging(hubgene_20,GTEX_sample_BL,GTEX_sample_B,GTEX_CPM_hsa,'Abnormal')


def CPM_norm(d):

    import bioinfokit.analys

    nm = bioinfokit.analys.norm()
    nm.cpm(df=d)
    
    cpm_gene = nm.cpm_norm
    
    return cpm_gene


#### GeneCPM_mfa (GeneCount - batch correction - CPM 계산)
file = './source_data/batch_correction/batch_cor_GeneCount_mfa.txt'
GeneCount_mfa = pd.read_csv(file, sep = '\t', header = 0)
GeneCount_mfa = GeneCount_mfa.set_index('gene_chr')


GeneCPM_mfa = pd.DataFrame()
for s in GeneCount_mfa.columns:
    df = GeneCount_mfa[[s]]
    df_cpm = CPM_norm(df)
    GeneCPM_mfa = pd.concat([GeneCPM_mfa,df_cpm],join='outer',axis=1) 

GeneCPM_mfa = np.log2(GeneCPM_mfa+1)


#### GeneCPM_mml
file = './source_data/count_result/GeneCount_mml.txt'
GeneCount_mml = pd.read_csv(file, sep = '\t', header = 0)
GeneCount_mml = GeneCount_mml.set_index('gene_chr')

GeneCPM_mml = pd.DataFrame()
for s in GeneCount_mml.columns:
    df = GeneCount_mml[[s]]
    df_cpm = CPM_norm(df)
    GeneCPM_mml = pd.concat([GeneCPM_mml,df_cpm],join='outer',axis=1) 

GeneCPM_mml = np.log2(GeneCPM_mml+1)


##### GeneCPM_condition
file = './source_data/reference/mml_condition.txt'
condition = pd.read_csv(file, sep = ' ', names=['sample','age','id','sex'])

In [ ]:
#GeneCPM_mfa
GeneCPM_MFA = GeneCPM_mfa.reset_index(drop=False)
GCM = pd.melt(GeneCPM_MFA, id_vars=['gene_chr'])
conditionlist = [GCM.variable.str.split('_').str[0] == 'A',
                 GCM.variable.str.split('_').str[0] == 'B',
                 GCM.variable.str.split('_').str[0] == 'C',
                 GCM.variable.str.split('_').str[0] == 'D',
                 GCM.variable.str.split('_').str[0] == 'E',
                 GCM.variable.str.split('_').str[0] == 'F',
                 GCM.variable.str.split('_').str[0] == 'G',
                 GCM.variable.str.split('_').str[0] == 'H']
choicelist = ['G1','G1','G2','G2','G3','G3','G4','G4']
GCM['Group'] = np.select(conditionlist,choicelist, default='Not Specified')

GCM['gene'] = GCM.gene_chr.str.split('_').str[0]
GCM_sel = GCM[GCM['gene'].isin(hubgene_20['gene'].unique())]

GCM_CPM = GCM_sel.groupby(['gene','Group'])['value'].mean()
GCM_CPM = GCM_CPM.reset_index(drop=False)
GCM_CPM = GCM_CPM.rename(columns={'value':'mfa_CPM'})
#print(GCM)

GCM_all = GCM.groupby(['gene','Group'])['value'].mean()
GCM_all = GCM_all.reset_index(drop=False)
GCM_all = GCM_all.rename(columns={'value':'mfa_logCPM'})
#print(GCM_all)

In [ ]:
#GeneCPM_mml
GeneCPM_MML = GeneCPM_mml.reset_index(drop=False)
GCL = pd.melt(GeneCPM_MML, id_vars=['gene_chr'])
GCL = GCL.rename(columns={'variable':'sample'})
GCL = pd.merge(GCL,condition,how='inner',on='sample')

conditionlist = [(5 >= GCL['age']) & (GCL['age'] >= 4),
                 (11 >= GCL['age']) & (GCL['age'] >= 7),
                 (19 >= GCL['age']) & (GCL['age'] >= 15),
                 (22 >= GCL['age']) & (GCL['age'] >= 21)]
choicelist = ['G1','G2','G3','G4']
GCL['Group'] = np.select(conditionlist,choicelist, default='Not Specified')
GCL = GCL[['gene_chr','sample','value','Group']]

GCL['gene'] = GCL.gene_chr.str.split('_').str[0]
GCL_sel = GCL[GCL['gene'].isin(hubgene_20['gene'].unique())]

GCL_CPM = GCL_sel.groupby(['gene','Group'])['value'].mean()
GCL_CPM = GCL_CPM.reset_index(drop=False)
GCL_CPM = GCL_CPM.rename(columns={'value':'mml_CPM'})

GCL_all = GCL.groupby(['gene','Group'])['value'].mean()
GCL_all = GCL_all.reset_index(drop=False)
GCL_all = GCL_all.rename(columns={'value':'mml_logCPM'})
#print(GCL_all)

In [ ]:
#Normal
GTEX_BL_CPM = GTEX_BL.groupby(['gene','AGE'])['value'].mean()
GTEX_BL_CPM = GTEX_BL_CPM.reset_index(drop=False)
GTEX_BL_CPM = GTEX_BL_CPM.rename(columns={'AGE':'Group','value':'hsa_logCPM'})
#print(GTEX_BL_CPM)

#Abnormal
GTEX_BL_Ab_CPM = GTEX_BL_Ab.groupby(['gene','AGE'])['value'].mean()
GTEX_BL_Ab_CPM = GTEX_BL_Ab_CPM.reset_index(drop=False)
GTEX_BL_Ab_CPM = GTEX_BL_Ab_CPM.rename(columns={'AGE':'Group','value':'hsa_logCPM'})

In [ ]:
mfa_mml_all = pd.merge(GCM_all,GCL_all,how='inner',on=['gene','Group'])
mfa_mml_all['age_group'] = mfa_mml_all['Group'].str.split('G').str[1]
mfa_mml_all['age_group'] = mfa_mml_all['age_group'].astype('int')

mfa_mml_all['hub'] = mfa_mml_all.apply(lambda x : 'hubgene' if x['gene'] in hubgene_20['gene'].unique().tolist() else 'gene', axis=1 )
mfa_mml_all = mfa_mml_all.sort_values(by=['hub'], ascending=[True])

r_1 = mfa_mml_all[mfa_mml_all['Group'] == 'G1']
r_2 = mfa_mml_all[mfa_mml_all['Group'] == 'G2']
r_3 = mfa_mml_all[mfa_mml_all['Group'] == 'G3']
r_4 = mfa_mml_all[mfa_mml_all['Group'] == 'G4']

r_1 = "R2: {:3.2f}".format(score_R2(r_1['mfa_logCPM'],r_1['mml_logCPM']))
r_2 = "R2: {:3.2f}".format(score_R2(r_2['mfa_logCPM'],r_2['mml_logCPM']))
r_3 = "R2: {:3.2f}".format(score_R2(r_3['mfa_logCPM'],r_3['mml_logCPM']))
r_4 = "R2: {:3.2f}".format(score_R2(r_4['mfa_logCPM'],r_4['mml_logCPM']))

plt.figure(figsize=(18,3))
sns.set(font_scale=1.5)
sns.set_style('white')
sns.lmplot(data=mfa_mml_all, x='mfa_logCPM', y='mml_logCPM',hue='hub',palette=['grey','lightgreen'],
           col='age_group',height=5, aspect=0.8, legend=False)

plt.text(-50, 0, r_1)
plt.text(-30, 0, r_2)
plt.text(-10, 0, r_3)
plt.text(10, 0, r_4)

sns.despine()

plt.show()

mfa_mml_all.to_csv('./source_data/hubgene/hubgenes_mfa_mml.txt',sep='\t',index=False,header=True)